In [1]:
import drdf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def load_drdf(fname):
  reader = drdf.DRDF()
  reader.read(fname)
  events = []
  for run in reader.runs:
    for event in reader.runs[run]:
      hits_map = dict()
      for cam, img in reader.runs[run][event].items():
        amplitude = img.pixels[:, :, 0] 
        time = img.pixels[:, :, 1]
        hits_map[cam] = amplitude
      events.append((event, hits_map))
  return events

#file = load_drdf("/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/response.drdf")

In [3]:
file_drdf_list = ["/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/initial_data_good/response.drdf","/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/new_data/response_pde25.drdf","/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/new_data2/response.drdf"]
file_root_list = ["/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/initial_data_good/sensors.root","/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/new_data/sensors_pde25.root","/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/new_data2/sensors.root"]

In [4]:
def CamList(fname):
    cam_list = []
    file = load_drdf(fname)
    for cam in file[0][1]:
        cam_list.append(cam)
    return cam_list

#cam_list = CamList()

In [5]:
#creo una matrice 1000 righe x 54 colonne. ogni elemento è una camera 31x31
def AllImages(fname): 
    all_images_list = []
    cam_list = CamList(fname)
    file = load_drdf(fname)
    for i in range(len(file)):
        ph_matrix = []
        for cam in cam_list:
            ph_matrix.append(file[i][1][cam])
        all_images_list.append(ph_matrix)
    all_images = np.array(all_images_list)
    return all_images

#all_images = AllImages()

In [6]:
def DimensionsData(fname):
    all_images = AllImages(fname)
    cam_list = CamList(fname)
    nr_events = len(all_images)
    nr_pixels = len(all_images[0][0])
    nr_cams = len(cam_list)
    nr_tot_events = nr_cams*nr_events
    nr_pixels_1_ev = nr_pixels*nr_pixels*nr_cams
    return nr_events, nr_pixels, nr_cams, nr_tot_events, nr_pixels_1_ev

In [7]:
#creo una matrice di 1000 righe e 51894 colonne. Così per ogni evento, la riga corrispondente presenta tutti i pixel di tutte le camere
def PixelsAllCamsPerEvents(fname):
    all_images = AllImages(fname)
    nr_events, nr_pixels, nr_cams, nr_tot_events, nr_pixels_1_ev = DimensionsData(fname)
    pixels_to_scale = []
    for images_all_cams_per_ev in all_images: 
        pixels_all_cams_per_ev = images_all_cams_per_ev.flatten()
        pixels_to_scale.append(pixels_all_cams_per_ev)
    pixels_to_scale = np.asarray(pixels_to_scale)
    pixels_to_scale_matrix = pixels_to_scale.reshape(nr_events,nr_pixels_1_ev,1)
    return pixels_to_scale_matrix

#pixels_to_scale_matrix = PixelsAllCamsPerEvents()

In [8]:
def PlotCamsImages(fname):
    all_images = AllImages(fname)
    file = load_drdf(fname)
    cam_list = CamList(fname)
    for i in range(len(all_images)):
        #for cam in cam_list:
        for j in range(len(all_images[i])):
            plt.imshow(all_images[i][j], interpolation='none')
            plt.colorbar()
            plt.title(f"event {file[i][0]} on {cam_list[j]}")
            plt.show()
            break
        break

#PlotCamsImages()

SCALING DATA

In [9]:
from sklearn.preprocessing import RobustScaler

#applico lo scaling su vettori colonne che sono le camere 31x31 appiattite
def ScalingData(fname):
    pixels_to_scale_matrix = PixelsAllCamsPerEvents(fname)
    nr_events, nr_pixels, nr_cams, nr_tot_events, nr_pixels_1_ev = DimensionsData(fname)
    photons_scaled_all_ev = []
    for pixels in pixels_to_scale_matrix:
        transformer = RobustScaler().fit(pixels)
        photons_scaled_in_cam = transformer.transform(pixels)
        photons_scaled_all_ev.append(photons_scaled_in_cam)
    photons_scaled_all_ev_matrix = np.asarray(photons_scaled_all_ev)
    all_images_scaled = photons_scaled_all_ev_matrix.reshape(nr_events,nr_cams,nr_pixels,nr_pixels)
    return all_images_scaled

#all_images_scaled = ScalingData()

In [10]:
#ora ho una matrice 1000righex54colonne dove un elemento è una matrice 31x31. Per essere consistente con i dati di root la voglio trasformare in un array di 54000 elementi
def Flattening(fname):
    all_images_scaled = ScalingData(fname)
    all_images_scaled_1d = []
    for sublist in all_images_scaled:
        all_images_scaled_1d.extend(sublist)
    return all_images_scaled_1d

#all_images_scaled_1d = np.array(Flattening())

In [11]:
def Reshaping(fname):
    all_images_scaled_1d_reshaped = []
    all_images = AllImages(fname)
    all_images_scaled_1d = Flattening(fname)
    nr_events, nr_pixels, nr_cams, nr_tot_events, nr_pixels_1_ev = DimensionsData(fname)
    for data in all_images_scaled_1d:
        data_new = data.reshape(nr_pixels,nr_pixels,1)
        all_images_scaled_1d_reshaped.append(data_new)
    return all_images_scaled_1d_reshaped

In [12]:
def PlotCamsImages(fname):
    all_images_scaled = ScalingData(fname)
    file = load_drdf(fname)
    cam_list = CamList(fname)
    for i in range(len(all_images_scaled)):
        for j in range(len(all_images_scaled[i])):
            plt.imshow(all_images_scaled[i][j], interpolation='none')
            plt.colorbar()
            plt.title(f"event {file[i][0]} on {cam_list[j]}")
            plt.show()
            break
        break

#PlotCamsImages()

In [13]:
def Preprocessing(fname):
    load_drdf(fname)
    CamList(fname)
    AllImages(fname)
    PixelsAllCamsPerEvents(fname)
    ScalingData(fname)
    Flattening(fname)
    return Reshaping(fname)

ROOT

In [14]:
import ROOT as root

def OpenRootFile(rname,fname):
    #sensor1.root è il file; ogni camera è un TTree
    input_file = root.TFile.Open(rname, "READ")
    #tree = input_file.Get("CAM_NB_X2")

    cam_list = CamList(fname)

    nr_photons_list_all_cams_list = []
    for cam in cam_list:
        nr_photons_list = []
        tree = input_file.Get(cam)
        entries = tree.GetEntries()
        for i in range(entries):
            n = tree.GetEntry(i)
            inner_photons = tree.innerPhotons
            nr_photons_list.append(inner_photons)
        nr_photons_list_all_cams_list.append(nr_photons_list)
        nr_photons_list_all_cams = np.array(nr_photons_list_all_cams_list)#ho creato una matrice 1000 colonne (nr eventi) x 54 righe (nr camere). I numeri che vediamo sono i numeri di fotoni che arrivano alla camera
    return nr_photons_list_all_cams

#nr_photons_list_all_cams = np.array(nr_photons_list_all_cams_list)#ho creato una matrice 1000 colonne (nr eventi) x 54 righe (nr camere). I numeri che vediamo sono i numeri di fotoni che arrivano alla camera


Welcome to JupyROOT 6.28/04


In [15]:
def EventListNumber(fname):
    file = load_drdf(fname)
    ev_list = []
    for i in range(len(file)):
        ev_list.append(file[i][0])
    return ev_list

In [16]:
def column(matrix, i):
    column = [row[i] for row in matrix]
    return column

def RootDataDictionary(rname,fname):
    nr_photons_list_all_cams = OpenRootFile(rname,fname)
    ev_list = EventListNumber(fname)
    all_photons_in_ev = []
    for i in ev_list:
        nr_photons_in_ev = column(nr_photons_list_all_cams, i)#lista di fotoni in un determinato evento/sono le colonne delle matrici
        all_photons_in_ev.append(nr_photons_in_ev)#faccio una lista di queste liste di fotoni

    cam_list = CamList(fname)

    final_root_data = []
    for i in range(len(all_photons_in_ev)):
        nr_photon_in_cam = []
        for j in range(len(all_photons_in_ev[i])):
            nr_photon_in_cam.append(all_photons_in_ev[i][j])
        dict_cam_ev = dict(zip(cam_list, nr_photon_in_cam))
        final_root_data.append((ev_list[i], dict_cam_ev))
    return final_root_data

In [17]:
def InnerPhotonsList(rname,fname):
    final_root_data = RootDataDictionary(rname, fname)
    inner_photons_list = []
    for i in range(len(final_root_data)):
        for cam in final_root_data[i][1].keys():
            inner_photons_list.append(final_root_data[i][1][cam])  
    inner_photons_list = np.asarray(inner_photons_list)
    return inner_photons_list

def CamStatesRootList(rname,fname):
    inner_photons_list = InnerPhotonsList(rname,fname)
    ev_cam_state_bool = inner_photons_list > 5
    ev_cam_state = ev_cam_state_bool.astype(int)
    return ev_cam_state

In [18]:
def RootPreprocessing(rname, fname):
    OpenRootFile(rname, fname)
    EventListNumber(fname)
    RootDataDictionary(rname, fname)
    return CamStatesRootList(rname, fname)

add other data

In [19]:
all_images_scaled_1d = Preprocessing(file_drdf_list[0])
ev_cam_state = RootPreprocessing(file_root_list[0],file_drdf_list[0])
#inner_photons_list = InnerPhotonsList("/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/initial_data/sensors.root","/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/data/initial_data/response.drdf")

In [20]:
#57 eventi e 58 camere, 32x32 pixels
data_prep2 = Preprocessing(file_drdf_list[1])
ev_cam_state2 = RootPreprocessing(file_root_list[1], file_drdf_list[1])

In [21]:
#57 eventi e 60 camere, 32x32 pixels 
data_prep3 = Preprocessing(file_drdf_list[2])
ev_cam_state3 = RootPreprocessing(file_root_list[2],file_drdf_list[2])

CNN MODEL

In [22]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from sklearn.model_selection import train_test_split

In [23]:
def SplitDataset(rname,fname):
    all_images_scaled_1d = Flattening(fname)
    inner_photons_list = InnerPhotonsList(rname,fname)
    #ev_cam_state = rooprep.CamStatesRootList(rname, fname)
    t_ds, val_ds, t_inner_ph, val_inner_ph = train_test_split(all_images_scaled_1d, inner_photons_list, train_size=0.9, random_state=42)
    train_ds, test_ds, train_inner_ph, test_inner_ph = train_test_split(t_ds, t_inner_ph, train_size=0.9, random_state=42)

    train_ds = np.asarray(train_ds)
    val_ds = np.asarray(val_ds)
    train_inner_ph = np.asarray(train_inner_ph) 
    val_inner_ph = np.asarray(val_inner_ph)
    test_ds = np.asarray(test_ds) 
    test_inner_ph = np.asarray(test_inner_ph)

    return train_ds, val_ds, test_ds, train_inner_ph, val_inner_ph, test_inner_ph


In [24]:
def PrepareDataSingleFile(rname, fname):
    train_ds, val_ds, test_ds, train_inner_ph, val_inner_ph, test_inner_ph = SplitDataset(rname,fname)
    nr_events, nr_pixels, nr_cams, nr_tot_events, nr_pixels_1_ev = DimensionsData(fname)

    train_labels_bool = train_inner_ph > 5
    train_labels = train_labels_bool.astype(int)

    val_labels_bool = val_inner_ph > 5
    val_labels = val_labels_bool.astype(int)

    test_labels_bool = test_inner_ph > 5
    test_labels = test_labels_bool.astype(int)

    train_ds = train_ds.reshape(train_ds.shape[0], nr_pixels,nr_pixels, 1)
    val_ds = val_ds.reshape(val_ds.shape[0], nr_pixels, nr_pixels, 1)
    test_ds = test_ds.reshape(test_ds.shape[0],nr_pixels,nr_pixels,1)
    return train_ds, val_ds, test_ds, train_labels, val_labels, test_labels

In [25]:
# #dataset1
# train_ds1, val_ds1, test_ds1, train_labels1, val_labels1, test_labels1 = PrepareDataSingleFile(file_root_list[0],file_drdf_list[0])

In [26]:
# #dataset2
# train_ds2, val_ds2, test_ds2, train_labels2, val_labels2, test_labels2 = PrepareDataSingleFile(file_root_list[1],file_drdf_list[1])

In [27]:
# #dataset3
# train_ds3, val_ds3, test_ds3, train_labels3, val_labels3, test_labels3 = PrepareDataSingleFile(file_root_list[2],file_drdf_list[2])

In [28]:
def WholeDataset(rname: list,fname: list):
    #dataset1
    train_ds1, val_ds1, test_ds1, train_labels1, val_labels1, test_labels1 = PrepareDataSingleFile(rname[0],fname[0])
    #dataset2
    train_ds2, val_ds2, test_ds2, train_labels2, val_labels2, test_labels2 = PrepareDataSingleFile(rname[1],fname[1])
    #dataset3
    train_ds3, val_ds3, test_ds3, train_labels3, val_labels3, test_labels3 = PrepareDataSingleFile(rname[2],fname[2])

    train_ds = np.concatenate((train_ds1, train_ds2, train_ds3), axis=0)
    val_ds = np.concatenate((val_ds1, val_ds2, val_ds3), axis=0)
    test_ds = np.concatenate((test_ds1, test_ds2, test_ds3), axis=0)

    train_labels = np.concatenate((train_labels1, train_labels2, train_labels3), axis=0)
    val_labels = np.concatenate((val_labels1, val_labels2, val_labels3), axis=0)
    test_labels = np.concatenate((test_labels1, test_labels2, test_labels3), axis=0)
    return train_ds, val_ds, test_ds,train_labels, val_labels,test_labels

In [29]:
def FindBlindEvents(rname,fname):
    ev_cam_state = RootPreprocessing(rname,fname)
    data_prep = Preprocessing(fname)
    n = 0
    nr_blind_events = []
    blind_events = []
    for i in ev_cam_state:
        if i == 1:
            nr_blind_events.append(n)
        n+=1
    for i in nr_blind_events:
        blind_events.append(data_prep[i])
    return nr_blind_events, blind_events

In [30]:
nr_blind_events1, blind_events1 = FindBlindEvents(file_root_list[0],file_drdf_list[0])
nr_blind_events2, blind_events2 = FindBlindEvents(file_root_list[1],file_drdf_list[1])
nr_blind_events3, blind_events3 = FindBlindEvents(file_root_list[2],file_drdf_list[2])

In [38]:
blind_events1_sub = []
for i in blind_events1:
    if np.max(i) > 100:
      blind_events1_sub.append(i)

In [39]:
def Augmentation(rname: list, fname: list):
    input_shape = [32,32,32,1]
    data_augmentation = keras.Sequential()
    data_augmentation.add(layers.RandomFlip("horizontal", input_shape=input_shape[1:]))
    data_augmentation.add(layers.RandomRotation(0.2))
    data_augmentation.add(layers.RandomTranslation(0.2,0.2))
    
    #nr_blind_events1, blind_events1 = FindBlindEvents(rname[0], fname[0])
    nr_blind_events2, blind_events2 = FindBlindEvents(rname[1], fname[1])
    nr_blind_events3, blind_events3 = FindBlindEvents(rname[2], fname[2])
    
    all_blind_events = np.concatenate((blind_events1_sub,blind_events2,blind_events3), axis=0)

    augmented_ds = []
    for blind_im in all_blind_events:
        for i in range(6):
            augmented_image = data_augmentation(blind_im)
            augmented_image_sq = np.squeeze(augmented_image, axis=3)
            augmented_ds.append(augmented_image_sq)
    augmented_ds = np.asarray(augmented_ds)

    augmented_labels = len(augmented_ds)*[1]
    return augmented_ds, augmented_labels

In [40]:
def DatasetwithAugmentation(rname: list, fname: list):
    train_ds, val_ds, test_ds,train_labels, val_labels,test_labels = WholeDataset(rname, fname)
    augmented_ds, augmented_labels = Augmentation(rname, fname)
    train_ds_aug = np.concatenate((train_ds,augmented_ds),axis=0)
    train_labels_aug = np.concatenate((train_labels,augmented_labels),axis=0)
    return train_ds_aug, train_labels_aug

In [41]:
def DatasetWeights(rname: list, fname: list):
    nr_blind_events1, blind_events1 = FindBlindEvents(rname[0],fname[0])
    nr_blind_events2, blind_events2 = FindBlindEvents(rname[1],fname[1])
    nr_blind_events3, blind_events3 = FindBlindEvents(rname[2],fname[2])

    nr_events1, nr_pixels1, nr_cams1, nr_tot_events1, nr_pixels_1_ev1 = DimensionsData(fname[0])
    nr_events2, nr_pixels2, nr_cams2, nr_tot_events2, nr_pixels_1_ev2 = DimensionsData(fname[1])
    nr_events3, nr_pixels3, nr_cams3, nr_tot_events3, nr_pixels_1_ev3 = DimensionsData(fname[2])

    augmented_ds, augmented_labels = Augmentation(rname, fname)
    
    nr_blind_ev = len(nr_blind_events1) + len(nr_blind_events2) + len(nr_blind_events3) + len(augmented_ds)
    nr_tot_ev = nr_tot_events1 + nr_tot_events2 + nr_tot_events3 + len(augmented_ds)
    nr_not_blind_ev = nr_tot_ev - nr_blind_ev

    initial_bias = np.log(nr_blind_ev/nr_not_blind_ev)

    weights_0 = (1/nr_not_blind_ev)*(nr_tot_ev/2)
    weights_1 = (1/nr_blind_ev)*(nr_tot_ev/2)
    weights_classes = {0: weights_0, 1: weights_1}
    return initial_bias, weights_classes

In [42]:
def Training(rname: list, fname: list): 
    #initial_bias, weights_classes = DatasetWeights(rname,fname)
    input_shape = [32,32,32,1]
    #output_bias = keras.initializers.Constant(initial_bias)
    model = models.Sequential()
    #model.add(data_augmentation)
    model.add(layers.Conv2D(8,3,padding='same', activation='ReLU', input_shape=input_shape[1:]))
    model.add(layers.Conv2D(16,3,padding='same', activation='ReLU', input_shape=input_shape[1:]))
    model.add(layers.Conv2D(16,3,padding='same', activation='ReLU', input_shape=input_shape[1:]))
    #model.add(layers.Conv2D(32,3,padding='same', activation='sigmoid', input_shape=input_shape[1:]))
    model.add(layers.MaxPooling2D((32,32)))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))#, bias_initializer=output_bias))

    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
    model.summary()
    return model


In [43]:
model = Training(file_root_list, file_drdf_list)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 8)         80        
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 16)        1168      
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2  (None, 1, 1, 16)          0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                        

In [44]:
train_ds, val_ds, test_ds,train_labels, val_labels,test_labels = WholeDataset(file_root_list, file_drdf_list)
train_ds_aug, train_labels_aug = DatasetwithAugmentation(file_root_list,file_drdf_list)
initial_bias, weights_classes = DatasetWeights(file_root_list, file_drdf_list)
epochs = 10

In [45]:
#callbacks = [keras.callbacks.ModelCheckpoint("/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/ultima_versione_buoni/tf_model_new/cams_model_new.keras")]
history = model.fit(train_ds_aug, train_labels_aug, validation_data= (val_ds, val_labels), epochs=epochs, batch_size=32, class_weight=weights_classes)

Epoch 1/10
1656/1656 [==============================] - 21s 13ms/step - loss: 2.0551 - accuracy: 0.9098 - val_loss: 0.4330 - val_accuracy: 0.9249
Epoch 2/10
1656/1656 [==============================] - 19s 11ms/step - loss: 0.2957 - accuracy: 0.9579 - val_loss: 0.1379 - val_accuracy: 0.9839
Epoch 3/10
1656/1656 [==============================] - 20s 12ms/step - loss: 0.8323 - accuracy: 0.9064 - val_loss: 0.9595 - val_accuracy: 0.9081
Epoch 4/10
1656/1656 [==============================] - 21s 13ms/step - loss: 0.9908 - accuracy: 0.9182 - val_loss: 0.6895 - val_accuracy: 0.9455
Epoch 5/10
1656/1656 [==============================] - 21s 12ms/step - loss: 0.6065 - accuracy: 0.9384 - val_loss: 0.1818 - val_accuracy: 0.9795
Epoch 6/10
1656/1656 [==============================] - 21s 13ms/step - loss: 0.5063 - accuracy: 0.9558 - val_loss: 0.1627 - val_accuracy: 0.9829
Epoch 7/10
1656/1656 [==============================] - 20s 12ms/step - loss: 1.1511 - accuracy: 0.9180 - val_loss: 0.7708 -

In [37]:
# train_ds, val_ds, test_ds,train_labels, val_labels,test_labels = WholeDataset(file_root_list, file_drdf_list)
# results = model.evaluate(test_ds,test_labels)

In [46]:
saved_model = model.save("/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/ultima_versione_buoni/tf_model_new/cams_model_aug2.keras")